In [1]:
# Importing dependencies 
import gmaps
from config import gkey
import pandas as pd
import requests

# Configuring gmaps with my Google API key
gmaps.configure(api_key=gkey)

In [2]:
# Setting the path to the data created in part I and reading it into a dataframe
path = "Resources/city_data.csv"
city_df = pd.read_csv(path)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.190000,168.860000,35.01,91.0,96.0,10.00,NZ,1.601212e+09
1,cape town,-33.925839,18.423218,60.80,44.0,40.0,5.82,ZA,1.601212e+09
2,xucheng,20.350001,110.128837,80.60,100.0,100.0,2.24,CN,1.601212e+09
3,inhambane,-23.865000,35.383333,84.20,66.0,40.0,6.93,MZ,1.601212e+09
4,lata,40.160000,-8.330000,77.00,52.0,0.0,7.23,PT,1.601212e+09


In [3]:
# Setting the heatmap locations to the latitude and longitude of each city in the dataset
locations = city_df[["Lat", "Lng"]]

# Assigning the weighting of the heatmap to the humidity of the city
rating = city_df["Humidity"].astype(float)

In [4]:
# Use the Gmaps module to load the figure
fig = gmaps.figure()

# Set up the heatmap with the heatmap_layer function
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.1)

# Adding the heatmat layer to the figure 
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Filtering the dataframe to only include weather conditions which are ideal
hotel_df = city_df.loc[(city_df["Max Temp"] > 90) & (city_df["Cloudiness"] < 10) & (city_df["Wind Speed"] < 5), :]

# Removing any NaN values from the dataframe
hotel_df = hotel_df.dropna()

hotel_df["Hotel Name"] = ""

# Checking how many remaining cities there are to choose from and previewing the data
print(f"The number of possible vacation destinations is: {len(hotel_df)}")
hotel_df

The number of possible vacation destinations is: 15


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
152,dire,12.283333,-10.966667,91.00,54.0,0.0,2.73,ML,1.601212e+09,
174,shakawe,-18.366667,21.850000,105.46,7.0,0.0,2.24,BW,1.601212e+09,
207,kidal,18.441111,1.407778,101.03,10.0,0.0,3.83,ML,1.601212e+09,
311,menongue,-14.661639,17.688224,106.02,7.0,0.0,1.39,AO,1.601212e+09,
432,binga,-17.620000,27.340000,100.04,15.0,0.0,2.82,ZW,1.601212e+09,
436,susangerd,31.560800,48.183100,111.20,14.0,0.0,4.70,IR,1.601212e+09,
539,nokaneng,-19.666667,22.266667,100.90,8.0,0.0,3.62,BW,1.601212e+09,
630,ibra,22.690278,58.530000,100.44,10.0,0.0,4.76,OM,1.601212e+09,
740,ajdabiya,30.759167,20.230000,98.02,24.0,1.0,3.36,LY,1.601212e+09,
893,khani,41.960000,42.960000,91.40,27.0,5.0,4.70,GE,1.601212e+09,


In [6]:
# Creating the API call to find the nearest hotel within 5000m of the ideal cities
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params = {
        "location": f"{row[1]}, {row[2]}",
        "keyword": "Hotel",
        "radius": 5000,
        "key": gkey,
    }
    
    city_name = row["City"]
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")

Retrieving Results for Index 152: dire.
Missing field/result... skipping.
------------
Retrieving Results for Index 174: shakawe.
Closest hotel to shakawe is Camp Roi.
------------
Retrieving Results for Index 207: kidal.
Missing field/result... skipping.
------------
Retrieving Results for Index 311: menongue.
Closest hotel to menongue is Ritz Lauca.
------------
Retrieving Results for Index 432: binga.
Closest hotel to binga is Tuyuni Guest House.
------------
Retrieving Results for Index 436: susangerd.
Closest hotel to susangerd is Hall, Hotel And Restaurant Ghasr.
------------
Retrieving Results for Index 539: nokaneng.
Missing field/result... skipping.
------------
Retrieving Results for Index 630: ibra.
Closest hotel to ibra is Ibra Hotel.
------------
Retrieving Results for Index 740: ajdabiya.
Closest hotel to ajdabiya is Almadina Hotel.
------------
Retrieving Results for Index 893: khani.
Closest hotel to khani is Nino village.
------------
Retrieving Results for Index 926: 

In [7]:
# For the rows where the hotel data was not found, drop those rows from the dataframe
NaN = float("NaN")
hotel_df.replace("", NaN, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

# Display the final dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
174,shakawe,-18.366667,21.850000,105.46,7.0,0.0,2.24,BW,1.601212e+09,Camp Roi
311,menongue,-14.661639,17.688224,106.02,7.0,0.0,1.39,AO,1.601212e+09,Ritz Lauca
432,binga,-17.620000,27.340000,100.04,15.0,0.0,2.82,ZW,1.601212e+09,Tuyuni Guest House
436,susangerd,31.560800,48.183100,111.20,14.0,0.0,4.70,IR,1.601212e+09,"Hall, Hotel And Restaurant Ghasr"
630,ibra,22.690278,58.530000,100.44,10.0,0.0,4.76,OM,1.601212e+09,Ibra Hotel
740,ajdabiya,30.759167,20.230000,98.02,24.0,1.0,3.36,LY,1.601212e+09,Almadina Hotel
893,khani,41.960000,42.960000,91.40,27.0,5.0,4.70,GE,1.601212e+09,Nino village
926,senanga,-16.116667,23.266667,109.02,7.0,2.0,1.05,ZM,1.601212e+09,Zambia
1222,ondjiva,-17.066667,15.733333,93.20,15.0,0.0,4.70,AO,1.601212e+09,Águia Verde Hotel
1368,iaciara,-14.100000,-46.630000,92.97,21.0,0.0,3.85,BR,1.601212e+09,Camargos Hotel


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Create the symbol layer
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

# Use the Gmaps module to load the figure
fig = gmaps.figure()

# Adding the heatmap layer and marker layer
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))